In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import os
from os.path import basename
import urllib
import shutil
from geopandas import gpd
from shapely.geometry import Point, Polygon
# import shapefile

# my script
from w210_attribute_library import withinstates, haversine_distance

datdir = "../data/"
attrs = "../attrs/"
modeld = "../model/"

/Users/carlosmoreno/opt/anaconda3/envs/w210/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf = gpd.read_file("../data/sink_density_classified_polys_1km_conus/sink_density_classified_polys_1km_conus.shp")
# geo_df = GeoDataFrame(df, crs=crs, geometry=geometry)

In [3]:
new_gdf = gdf.to_crs(epsg=4326)
new_gdf.head(1)

,Id,gridcode,Shape_Leng,Shape_Area,geometry
0,1,1,6000.0,2.000000e+06,"POLYGON ((-122.15934 48.91415, -122.17231 48.9..."


In [4]:
new_gdf['x_coord'] = new_gdf.apply(lambda row: row['geometry'].centroid.x, axis=1)
new_gdf['y_coord'] = new_gdf.apply(lambda row: row['geometry'].centroid.y, axis = 1)

In [6]:
new_gdf.to_csv(datdir+"karst_gridcode_raw_data.csv", index=False)

### Read Karst Data
Note: if already created, load file here

In [7]:
fkarst = 'karst_gridcode_raw_data.csv'
dfk = pd.read_csv(datdir+fkarst)
print(len(dfk))
dfk.head(1)

10311


,Id,gridcode,Shape_Leng,Shape_Area,geometry,x_coord,y_coord
0,1,1,6000.0,2.000000e+06,POLYGON ((-122.15934291708528 48.9141451238035...,-122.169494,48.921667


### Select Relevant Geopoints

In [8]:
subdir = "../data/shapefile/"
shapedir = 'cb_2018_us_state_500k/'
shapefile500 = "cb_2018_us_state_500k.shp"

us500 = gpd.read_file(subdir+shapedir+shapefile500)

flgeometry = list(us500[(us500["NAME"]=='Florida')]["geometry"])[0]
gageometry = list(us500[(us500["NAME"]=='Georgia')]["geometry"])[0]
algeometry = list(us500[(us500["NAME"]=='Alabama')]["geometry"])[0]

geometries = [flgeometry, gageometry, algeometry ]
# geometries = [flgeometry]

# dfk["Florida"] = dfk.apply(lambda row: "FL" if (Point(row["x_coord"],row["y_coord"]).within(flgeometry)) else "NoFL", axis=1)

dfk["in_relevant_state"] = dfk.apply(lambda row: withinstates(geometries, Point(row["x_coord"],row["y_coord"])), axis=1)

dfk = dfk[dfk["in_relevant_state"] == "Yes"]
len(dfk)

2021

### Read Tile Data

In [10]:
ftile = 'model_satel_attr_365.csv'
dtile = pd.read_csv(datdir+ftile)
dtile.columns

Index(['name', 'imgnum', 'label', 'ID', 'lon', 'lat', 'start_date', 'geometry',
       'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
       'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake',
       'prediction', 'prediction_name', 'Group', 'Key'],
      dtype='object')

In [11]:
dtile = dtile[['Key', 'lon', 'lat']]

#### Cross Merge Dataframes

In [12]:
dtileK = pd.merge(dtile, dfk, how="cross")
print(len(dtileK))
dtileK.head(1)

681077


,Key,lon,lat,Id,gridcode,Shape_Leng,Shape_Area,geometry,x_coord,y_coord,in_relevant_state
0,2012_0_1,-81.399778,30.24471,7853,1,8000.0,3000000.0,POLYGON ((-85.81425135523526 34.95544180695513...,-85.814703,34.965969,Yes


#### Calculate Distance

In [14]:
dtileK['Distance'] = dtileK.apply(lambda row: 
                                    haversine_distance(row['lat'], row['lon'], 
                                                       row['y_coord'], row['x_coord'], 
                                                       earth_radius=3963.19), axis=1)
dtileK.head(2)

,Key,lon,lat,Id,gridcode,Shape_Leng,Shape_Area,geometry,x_coord,y_coord,in_relevant_state,Distance
0,2012_0_1,-81.399778,30.24471,7853,1,8000.0,3.000000e+06,POLYGON ((-85.81425135523526 34.95544180695513...,-85.814703,34.965969,Yes,415.616881
1,2012_0_1,-81.399778,30.24471,7899,1,8000.0,3.000000e+06,POLYGON ((-85.34507422607149 34.81466620602329...,-85.345461,34.825197,Yes,391.483297


## Find the Minimum Distances within Tile and Weather Station

`df.groupby('Company')['MPG'].agg('min')`  
`df.groupby('Company')[['MPG', 'EngineSize']].agg('min')`

**Reference:**
https://datascienceparichay.com/article/pandas-groupby-minimum/

In [15]:
# Find the rows with the minimum Distance for each Key_x
dfmin1 = dtileK.groupby(['Key'])['Distance'].min().to_frame()
print(len(dfmin1))

# Select only the rows with the minimum
keysL = list(dfmin1.index)
minD = list(dfmin1['Distance'])
dfF1 = dtileK[((dtileK['Key'].isin(keysL)) &  (dtileK['Distance'].isin(minD)))]

337


In [16]:
#Checking for Duplicates
df2 = dfF1[dfF1["Key"].duplicated()==True]
dup1 = df2["Key"].unique()
# dup1 = ['1082_0_1','1083_0_1', '2406_0_1', '2459_0_1', '2463_0_1', '2737_0_1', '3294_0_2', '3294_1_0', '556_0_1']
dfF1[(dfF1['Key'].isin(dup1))]

,Key,lon,lat,Id,gridcode,Shape_Leng,Shape_Area,geometry,x_coord,y_coord,in_relevant_state,Distance


In [17]:
dfF1.drop_duplicates(subset=['Key'], inplace=True)
dfF1["Distance"].describe()

/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_24512/3709211602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfF1.drop_duplicates(subset=['Key'], inplace=True)


count    337.000000
mean       9.270980
std       14.015003
min        0.016013
25%        1.471366
50%        4.243829
75%       10.730072
max      132.931690
Name: Distance, dtype: float64

In [18]:
dfF1["gridcode"].unique()

array([1, 2, 0, 3])

In [19]:
dfF1.columns

Index(['Key', 'lon', 'lat', 'Id', 'gridcode', 'Shape_Leng', 'Shape_Area',
       'geometry', 'x_coord', 'y_coord', 'in_relevant_state', 'Distance'],
      dtype='object')

In [20]:
dfF1 = dfF1[['Key', 'gridcode']]

In [21]:
dfF1.to_excel(datdir+"w210_karst.xlsx", index=False)